In [1]:
#!pip install "transformers==4.31.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1" --upgrade

In [1]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files="PlantQA.csv")

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 1002.94it/s]


Dataset csv downloaded and prepared to /home/CE/alecakir/.cache/huggingface/datasets/csv/default-f205dd177ba9dfb4/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 698.58it/s]


In [10]:
dataset = dataset["train"]

In [11]:
from random import randrange


print(f"dataset size: {len(dataset)}")
print(dataset[randrange(len(dataset))])


dataset size: 184
{'Unnamed: 0': 1339, 'id': 't3_7y1xyd', 'title': 'My cactus has been suffering from a lack of light lately so I started leaving him under a plant light when it’s cloudy out. He recently stated getting these weird scabby spots. They don’t seem to be spreading too fast, if at all. Does anyone know what they are?', 'comment': 'Looks like it could be scale.'}


In [12]:
from datasets import Dataset
my_dict = dataset[:32]
dataset = Dataset.from_dict(my_dict)

In [13]:
def format_instruction(sample):
	return f"""### Instruction:
Use the Input below to create an instruction, which could have been used to generate the input using an LLM.

### Input:
{sample['title']}

### Response:
{sample['comment']}
"""


In [14]:
from random import randrange

print(format_instruction(dataset[randrange(len(dataset))]))

### Instruction:
Use the Input below to create an instruction, which could have been used to generate the input using an LLM.

### Input:
My poor hobbit plant! He is turning all gooey and falling apart. Seems to be doing from the stem up. I know he got a little scorched so I moved him farther away from the morning sun. He had yellow mushrooms pop up, and they came to full head before I could get them out. Is there any saving him?

### Response:
That sounds like over watering to me. I would check to make sure there is adequate drainage in that pot, the earth looks wet. And, I would move him back into the sun after a change of soil, and some prunning of dead mushy stuff off.



In [15]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

use_flash_attention = False


# Hugging Face model id
model_id = "NousResearch/Llama-2-7b-hf" # non-gated
# model_id = "meta-llama/Llama-2-7b-hf" # gated


# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, use_cache=False, device_map="auto")
model.config.pretraining_tp = 1

# Validate that the model is using flash attention, by comparing doc strings
if use_flash_attention:
    from utils.llama_patch import forward
    assert model.model.layers[0].self_attn.forward.__doc__ == forward.__doc__, "Model is not using flash attention"


tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.70s/it]


In [16]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# LoRA config based on QLoRA paper
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM",
)


# prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)


In [17]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="llama-7-int4-dolly",
    num_train_epochs=1,
    per_device_train_batch_size=6 if use_flash_attention else 1,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    fp16=True,
    tf32=False,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    disable_tqdm=True # disable tqdm since with packing values are in correct
)


In [18]:
from trl import SFTTrainer

max_seq_length = 2048 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=format_instruction,
    args=args,
)


In [20]:
# train
trainer.train() # there will not be a progress bar since tqdm is disabled

# save model
trainer.save_model()


{'train_runtime': 49.84, 'train_samples_per_second': 0.642, 'train_steps_per_second': 0.321, 'train_loss': 2.3799829483032227, 'epoch': 0.06}


In [21]:
if use_flash_attention:
    # unpatch flash attention
    from utils.llama_patch import unplace_flash_attn_with_attn
    unplace_flash_attn_with_attn()

import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

args.output_dir = "llama-7-int4-dolly"

# load base LLM model and tokenizer
model = AutoPeftModelForCausalLM.from_pretrained(
    args.output_dir,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)
tokenizer = AutoTokenizer.from_pretrained(args.output_dir)


Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.57s/it]


In [24]:
from datasets import load_dataset
from random import randrange


# Load dataset from the hub and get a sample
#dataset = load_dataset("databricks/databricks-dolly-15k", split="train")
sample = dataset[randrange(len(dataset))]

prompt = f"""### Instruction:
Use the Input below to create an instruction, which could have been used to generate the input using an LLM.

### Input:
{sample['title']}

### Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.9)

print(f"Prompt:\n{sample['title']}\n")
print(f"Generated instruction:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"Ground truth:\n{sample['comment']}")


Prompt:
Purchased this Crassula Buddha’s Temple with brown leaves, what should I do to ensure it thrives when it arrives?

Generated instruction:
Brown leaves are normal for this type of succulent. If they're turning yellow or starting to fall off the plant, the plant might not have enough sunlight or water. If you have brown leaves, you should trim them off the plant.

### Explanation:
- [https://imgur.com/a/49nR7QG](https://imgur.com/a/49nR7QG)
- [https://
Ground truth:
maybe check in with /r/succulents and see what they have to say about. But I have a couple and treat them the same as my other drier succulents and cacti. 

Since I got them a few months ago I've watered maybe three times? I have them under a grow light and they've been doing fine. Once it gets warmer and stays warmer I'll probably begin to water more frequently to compensate. 

You can also check here:
https://worldofsucculents.com/grow-and-care-for-crassula/

For more growing information. 
